## Bronze

this will be the bronze section. Basically ingesting data.

In [ ]:
import pandas as pd

# List of Parquet files
files = [
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\ListaCobroDetalle2022.parquet",
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\ListaCobroDetalle2023.parquet",
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\ListaCobroDetalle2024.parquet",
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\ListaCobroDetalle2025.parquet"
]

# Read and concatenate into one DataFrame
df_append = pd.concat([pd.read_parquet(file) for file in files], ignore_index=True)


In [ ]:
df_bronze = df_append.copy()

##### Adding Catalogs.

In [ ]:
cat_banco = pd.read_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\CatBanco.parquet",)
cat_emisora = pd.read_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\CatEmisora.parquet",)
# Read the Parquet file
cat_respuestabanco = pd.read_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\CatRespuestaBancos.parquet",)
# Read the Parquet file
catlistemisora = pd.read_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\ListaCobroEmisora.parquet",)
cat_listacobro = pd.read_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\ListaCobro.parquet",)
df_newclients = pd.read_csv(r"C:\Users\ibarrj40\Downloads\MontoCapital_Pagare_FechaAperturaCredito(Hoja1).csv", encoding='latin1')
cat_emisora_horarios = pd.read_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\banco_emisoras_horarios.parquet",
)   



# Silver

This is for making transformations, joins and castings.


#### Merging with the catalog Banco

In [ ]:
cat_banco.head()
# Merge the DataFrame with the catalog

In [ ]:
cat_banco.rename(columns={'IdBanco': 'idBanco'}, inplace=True)

In [ ]:
df_append = df_append.merge(cat_banco, on='idBanco', how='left')
df_append = df_append.rename(columns={'Nombre': 'Banco'})

### Adding Respuestas Banco to the df

In [ ]:


cat_respuestabanco = cat_respuestabanco.rename(columns={'IdRespuestaBanco': 'idRespuestaBanco'})

In [ ]:
cat_respuestabanco.head()

In [ ]:
df_append = df_append.merge(cat_respuestabanco, on='idRespuestaBanco',  how='left')

### Adding Cat_emisora

In [ ]:
cat_emisora = cat_emisora.rename(columns={'IdBanco': 'idBanco'})
cat_emisora['idBanco'] = cat_emisora['idBanco'].astype(str)
cat_banco['idBanco'] = cat_banco['idBanco'].astype(str)

In [ ]:
cat_emisora_wb = cat_emisora.merge(cat_banco, on='idBanco', how='left')


In [ ]:

cat_emisora_wb.rename(columns={'Nombre_x': 'Nombre_Emisora','Nombre_y': 'Nombre_Banco_Emisora', 'idBanco': 'idBanco_Emisora'}, inplace=True) 
cat_emisora_wb.head()


### Adding Lista Cobro Emisora with dates

In [ ]:

catlistemisora.head()



In [ ]:
catlistemisora['idEmisora'] = catlistemisora['idEmisora'].astype(str)
cat_emisora_wb['idEmisora'] = cat_emisora_wb['idEmisora'].astype(str)


cat_listacobroemisora = catlistemisora.merge(cat_emisora_wb, on='idEmisora', how='left')

In [ ]:
cat_listacobroemisora.head()

In [ ]:
df_append = df_append.merge(cat_listacobroemisora, on='idListaCobro', how='left')

In [ ]:
df_append.head()

In [ ]:
df_append = df_append.merge(cat_respuestabanco, on='idRespuestaBanco', how='left')

In [ ]:
df_append.drop(columns=['Descripcion_y'], inplace=True)
df_append.rename(columns={'Descripcion_x': 'Descripcion'}, inplace=True)

#### Merging with lista cobro

In [ ]:
df_append = df_append.merge(cat_listacobro, on='idListaCobro', how='left')

In [ ]:
df_append.head()

In [ ]:
df_append.drop(columns=['idBanco_y'], inplace=True, errors='ignore')


In [ ]:
df_append.drop_duplicates()

## Applying the Schema to the dataframe

In [ ]:
df_append['idListaCobro'] = pd.to_numeric(df_append['idListaCobro'], errors='coerce')
df_append['idCredito'] = pd.to_numeric(df_append['idCredito'], errors='coerce')
df_append['consecutivoCobro'] = pd.to_numeric(df_append['consecutivoCobro'], errors='coerce')
df_append['montoExigible'] = pd.to_numeric(df_append['montoExigible'], errors='coerce')
df_append['montoCobrar'] = pd.to_numeric(df_append['montoCobrar'], errors='coerce')
df_append['montoCobrado'] = pd.to_numeric(df_append['montoCobrado'], errors='coerce')
df_append['idRespuestaBanco'] = df_append['idRespuestaBanco'].astype('string')
df_append['fechaCobroBanco'] = pd.to_datetime(df_append['fechaCobroBanco'], errors='coerce')
df_append['Banco'] = df_append['Banco'].astype('string')
df_append['Descripcion_y'] = df_append['Descripcion_y'].astype('string')
df_append['idEmisora'] = pd.to_numeric(df_append['idEmisora'], errors='coerce')
df_append['Nombre_y'] = df_append['Nombre_y'].astype('string')
df_append['IdBanco'] = pd.to_numeric(df_append['IdBanco'], errors='coerce')
df_append['TipoEnvio'] = df_append['TipoEnvio'].astype('string')


In [ ]:
df_newclients.rename(columns={
    'Crédito': 'IDCREDITO',
    'Capital Inicial': 'CAPITAL_INICIAL',
    'Pagaré': 'CREDITO_A_PAGAR',
    'FechaApertura': 'FECHA_APERTURA_CREDITO',
})

In [ ]:
cat_emisora_horarios = pd.read_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\banco_emisoras_horarios.parquet",
)   

# Cambiar tipo de la columna 'EMISORA' a string
cat_emisora_horarios['EMISORA'] = cat_emisora_horarios['EMISORA'].astype('string')

# Convertir las columnas de hora a datetime.time
cat_emisora_horarios['INICIO HORARIO LIMITE CARGA BANCARIA'] = pd.to_datetime(
    cat_emisora_horarios['INICIO HORARIO LIMITE CARGA BANCARIA'], format='%H:%M'
).dt.time

cat_emisora_horarios['FIN HORARIO LIMITE CARGA BANCARIA'] = pd.to_datetime(
    cat_emisora_horarios['FIN HORARIO LIMITE CARGA BANCARIIA'], format='%H:%M'
).dt.time

cat_emisora_horarios.rename(columns={
    'INICIO HORARIO LIMITE CARGA BANCARIA': 'inicio_horario_limite_carga_bancaria',
    'FIN HORARIO LIMITE CARGA BANCARIA': 'fin_horario_limite_carga_bancaria',
    'EMISORA': 'Emisora'
}, inplace=True)
# Merge con el DataFrame principal

df_append.rename(columns={'Emisora_x': 'Emisora'}, inplace=True)
cat_emisora_horarios['Emisora'] = cat_emisora_horarios['Emisora'].astype('string')

print(cat_emisora_horarios.columns)
df_append = df_append.merge(cat_emisora_horarios, left_on='Emisora', right_on='Emisora', how='left')

In [ ]:
df_newclients['IDCREDITO'] = pd.to_numeric(df_newclients['IDCREDITO'], errors='coerce')
df_newclients['FECHA_APERTURA_CREDITO'] = pd.to_datetime(df_newclients['FECHA_APERTURA_CREDITO'], errors='coerce')
df_newclients['CAPITAL_INICIAL'] = pd.to_numeric(
    df_newclients['CAPITAL_INICIAL'].str.replace(',', '', regex=False),
    errors='coerce'
)
df_newclients['CREDITO_A_PAGAR'] = pd.to_numeric(
    df_newclients['CREDITO_A_PAGAR'].str.replace(',', '', regex=False),
    errors='coerce'
)

In [ ]:
# Elimina columnas duplicadas (mantiene la primera aparición)
df_append = df_append.loc[:, ~df_append.columns.duplicated()]


In [ ]:
df_append.columns = df_append.columns.str.upper()

In [ ]:
df_silver = df_append.copy()

# Gold

Gold is for user business data, ready to consume with new aggregated values

In [ ]:
df_gold = df_append.copy()

In [ ]:
df_gold["isSuccessful"] = df_gold["MONTOCOBRADO"].ne(0)

In [ ]:
df_gold.head()

In [ ]:
df_gold_newclients = df_gold.merge(
    df_newclients,
    left_on='IDCREDITO',
    right_on='IDCREDITO',
    how='inner'
)

In [ ]:
df_gold_newclients = df_gold_newclients[df_gold_newclients['CAPITAL_INICIAL'].notna()]

In [ ]:
df_gold_newclients 

In [ ]:
df_gold['FECHACREACIONLISTA'] = pd.to_datetime(df_gold['FECHACREACIONLISTA'], errors='coerce', dayfirst=True)
df_gold = df_gold.sort_values(by=['IDCREDITO', 'FECHACREACIONLISTA'], ascending=False)
df_gold['INTENDODECOBRO'] = df_gold.groupby('IDCREDITO').cumcount() + 1
df_gold['INTENDODECOBRO'] = df_gold['INTENDODECOBRO'].astype('Int64')
df_gold = df_gold[df_gold["SERVICIO"].isnull()]
df_gold.drop(columns=['SERVICIO', 'CANAL DE ENVIO', 'TIPO ENVIO', 'COBROS'], inplace=True, errors='ignore')
df_gold.drop(columns=['FECHACOBROBANCO', 'FECHACREACIONLISTA'], inplace=True, errors='ignore')
df_gold.drop(columns=['INICIO_HORARIO_LIMITE_CARGA_BANCARIA', 'FIN HORARIO LIMITE CARGA BANCARIIA', 'IDBANCO_EMISORA'], inplace=True, errors='ignore')

#fill nulls
df_gold['DESCRIPCION'] = df_gold['DESCRIPCION'].fillna('Domiciliacion Exitosa').astype(str)


In [ ]:
df_checkemisoras_gold = df_gold.groupby(
    ['NOMBRE_EMISORA'],
    as_index=False
).agg({
    'MONTOCOBRADO': 'sum',
    'MONTOEXIGIBLE': 'sum',
    'MONTOCOBRAR': 'sum',
})

In [ ]:

import numpy as np

df_checkemisoras_gold['Percentaje'] = df_checkemisoras_gold['MONTOCOBRADO'] / df_checkemisoras_gold['MONTOEXIGIBLE']
df_checkemisoras_gold['Percentaje'] = pd.to_numeric(df_checkemisoras_gold['Percentaje'], errors='coerce')
df_checkemisoras_gold['Percentaje'] = df_checkemisoras_gold['Percentaje'].replace([np.inf, -np.inf], np.nan)
df_checkemisoras_gold['Percentaje'] = df_checkemisoras_gold['Percentaje'].fillna(0)
df_checkemisoras_gold = df_checkemisoras_gold.sort_values(by='Percentaje', ascending=False)
df_checkemisoras_gold = df_checkemisoras_gold.reset_index(drop=True)


## To parquets

In [ ]:
df_bronze.to_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\bronze\bronze_2022_2025.parquet",
    index=False
)

In [ ]:
df_silver.to_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\silver\silver.parquet",
    index=False
)

In [ ]:
df_gold.to_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\gold\gold.parquet",
    index=False
)

In [ ]:
df_gold_newclients.to_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\gold\gold_newclients.parquet",
    index=False
)

In [ ]:
df_checkemisoras_gold.to_parquet(
    r"C:\Users\ibarrj40\OneDrive\Documents\GitHub\Credifiel\gold\checkemisoras_gold.parquet",
    index=False
)

# Insights

In [ ]:
# 1. Emisoras con al menos un pago exitoso
emisoras_con_exito = df_append[df_append['idCredito'].isin(creditos_con_exito)]['Nombre_x'].unique()

# 2. Emisoras que nunca fallaron
emisoras_nunca_fallaron = df_append[df_append['idCredito'].isin(creditos_nunca_fallaron)]['Nombre_x'].unique()

print("🟢 Emisoras cuyos créditos nunca fallaron:")
print(emisoras_nunca_fallaron)

print("\n🟡 Emisoras con al menos un pago exitoso:")
print(emisoras_con_exito)


In [ ]:
# Total de intentos por emisora
total_por_emisora = df_append.groupby('Nombre_x')['isSuccessful'].count()

# Fallos por emisora
fallos_por_emisora = df_append[df_append['isSuccessful'] == False].groupby('Nombre_x')['isSuccessful'].count()

# Porcentaje de fallos
porcentaje_fallos_emisora = (fallos_por_emisora / total_por_emisora) * 100
porcentaje_fallos_emisora = porcentaje_fallos_emisora.sort_values(ascending=False)

# Mostrar resultados
print("\n🔴 Porcentaje de fallos por emisora:")
display(porcentaje_fallos_emisora.round(2))

# Emisora con mayor porcentaje de fallos
emisora_peor = porcentaje_fallos_emisora.idxmax()
porcentaje_peor = porcentaje_fallos_emisora.max()

display(porcentaje_fallos_emisora)
print(f"\n❌ Emisora con mayor porcentaje de fallos: {emisora_peor} ({porcentaje_peor:.2f}%)")


In [ ]:
# Total de créditos únicos
total_creditos = df_append['idCredito'].nunique()

# Agrupar por idCredito y verificar si hay al menos un pago exitoso
con_al_menos_un_pago = df_append.groupby('idCredito')['isSuccessful'].any()

# Agrupar por idCredito y verificar si **todos** los pagos fueron exitosos
nunca_dejaron_de_pagar = df_append.groupby('idCredito')['isSuccessful'].all()

# Calcular porcentaje de cada uno
porcentaje_con_al_menos_un_pago = (con_al_menos_un_pago.sum() / total_creditos) * 100
porcentaje_nunca_fallaron = (nunca_dejaron_de_pagar.sum() / total_creditos) * 100

# Mostrar resultados
print(f"✅ Porcentaje de créditos con al menos un pago exitoso: {porcentaje_con_al_menos_un_pago:.2f}%")
print(f"🔁 Porcentaje de créditos que nunca dejaron de pagar: {porcentaje_nunca_fallaron:.2f}%")


In [ ]:
# Obtener los idCredito que tuvieron al menos un pago exitoso
creditos_con_exito = con_al_menos_un_pago[con_al_menos_un_pago == True].index

# Obtener los idCredito que nunca dejaron de pagar
creditos_nunca_fallaron = nunca_dejaron_de_pagar[nunca_dejaron_de_pagar == True].index

# Filtrar el DataFrame original por esos créditos y obtener los bancos involucrados
bancos_con_exito = df_append[df_append['idCredito'].isin(creditos_con_exito)]['Banco'].unique()
bancos_nunca_fallaron = df_append[df_append['idCredito'].isin(creditos_nunca_fallaron)]['Banco'].unique()

# Mostrar los bancos
print("🏦 Bancos con al menos un pago exitoso:")
display(bancos_con_exito)

print("\n🏦 Bancos cuyos créditos nunca fallaron en pagar:")
display(bancos_nunca_fallaron)


In [ ]:
# Total de intentos por banco
total_por_banco = df_append.groupby('Banco')['isSuccessful'].count()

# Total de fallos por banco (donde isSuccessful == False)
fallos_por_banco = df_append[df_append['isSuccessful'] == False].groupby('Banco')['isSuccessful'].count()

# Calcular porcentaje de fallos por banco
porcentaje_fallos = (fallos_por_banco / total_por_banco) * 100

# Ordenar de mayor a menor
porcentaje_fallos = porcentaje_fallos.sort_values(ascending=False)

# Mostrar resultados
print("📉 Porcentaje de fallos por banco:")
print(porcentaje_fallos.round(2))

# Banco con mayor porcentaje de fallos
banco_peor = porcentaje_fallos.idxmax()
porcentaje_peor = porcentaje_fallos.max()

print(f"\n❌ Banco con mayor porcentaje de fallos: {banco_peor} ({porcentaje_peor:.2f}%)")


In [ ]:
# Agrupar por banco y calcular tasa de éxito: promedio de isSuccessful
banco_success_rate = df_append.groupby('Banco')['isSuccessful'].mean().sort_values(ascending=False) * 100

# Mostrar todos los bancos con su tasa de éxito
print("📊 Porcentaje de éxito por banco:")
print(banco_success_rate.round(2))

# Banco con mayor éxito
banco_top = banco_success_rate.idxmax()
porcentaje_top = banco_success_rate.max()

print(f"\n🏆 Banco con mayor éxito: {banco_top} ({porcentaje_top:.2f}%)")


In [ ]:
# Agrupar por día y mes, calcular porcentaje de éxito
df_append['day'] = df_append['fechaEnvioCobro'].dt.day
df_append['month'] = df_append['fechaEnvioCobro'].dt.month

# Filtrar solo fechas válidas
# Porcentaje de éxito por día
success_by_day = (
    df_append.groupby('day')['isSuccessful']
    .mean()
    .reset_index()
    .rename(columns={'isSuccessful': 'success_rate'})
    .sort_values('success_rate', ascending=False)
)

display(success_by_day)

# Porcentaje de éxito por mes
success_by_month = (
    df_append.groupby('month')['isSuccessful']
    .mean()
    .reset_index()
    .rename(columns={'isSuccessful': 'success_rate'})
    .sort_values('success_rate', ascending=False)
)

display(success_by_month)

# DataFrame con meses y éxitos
df_month_success = (
    df_append.groupby('month')['isSuccessful']
    .sum()
    .reset_index()
    .rename(columns={'isSuccessful': 'success_count'})
)
display(df_month_success)


success_by_day, success_by_month, df_month_success